In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [23]:
dataset = pd.read_csv('amazonreviews.tsv', delimiter = '\t', quoting = 3)

In [24]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 10000):
  review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i]) # Remove all the punctuations
  review = review.lower()
  review = review.split() # Split it to different elements (words)
  ps = PorterStemmer() 
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 0].values

In [26]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
print(y)

['pos' 'pos' 'pos' ... 'neg' 'neg' 'pos']


In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [29]:
print(y)

[1 1 1 ... 0 0 1]


In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [31]:
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [33]:
print(X_train)

[[-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 ...
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815  5.73622459 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]]


# Building the Neaural Network

In [34]:
ann = tf.keras.models.Sequential()

In [35]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [36]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [37]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [38]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [39]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6856 - accuracy: 0.6141
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4351 - accuracy: 0.8303
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2533 - accuracy: 0.9011
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1651 - accuracy: 0.9423
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1139 - accuracy: 0.9631
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.0799 - accuracy: 0.9734
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.0565 - accuracy: 0.9821
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.0424 - accuracy: 0.9864
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.0320 - accuracy: 0.9909
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.0264 - accura

In [40]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 0]
 [1 1]
 [0 0]
 ...
 [1 1]
 [0 0]
 [1 1]]


In [41]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[835 162]
 [180 823]]


0.829